In [1]:
# Live Accelerometer Data Visualization Dashboard
import os
import time
import threading
from datetime import datetime
from collections import deque
from threading import Lock, Event
import pandas as pd
import numpy as np
from dash import Dash, dcc, html, Output, Input
import plotly.graph_objects as go
from arduino_iot_cloud import ArduinoCloudClient
from iot_secrets.secrets import DEVICE_ID, SECRET_KEY

# Configuration settings
max_samples_in_graph = 500        # Number of samples to display in the live graph
auto_save_interval = 1000         # Save CSV file after collecting this many samples
data_output_folder = "week8C_data"
os.makedirs(data_output_folder, exist_ok=True)

# Possible variable names for accelerometer data in Arduino Cloud
x_axis_variables = ["x", "py_x", "accelerometer_X", "Accelerometer_X", "accel_x"]
y_axis_variables = ["y", "py_y", "accelerometer_Y", "Accelerometer_Y", "accel_y"]
z_axis_variables = ["z", "py_z", "accelerometer_Z", "Accelerometer_Z", "accel_z"]

# Thread-safe shared data storage
data_lock = Lock()
shutdown_signal = Event()
accelerometer_history = deque(maxlen=20000)
total_samples_collected = 0
previous_graph_figure = None

# Helper function to generate timestamp for file names
def generate_timestamp_filename():
    return datetime.now().strftime("%Y%m%d-%H%M%S")

def add_complete_reading(axis_name, reading_value, current_readings):
    """Add a complete accelerometer reading when all three axes have been received."""
    with data_lock:
        current_readings[axis_name] = reading_value
        # Only add to history when we have all three axis values
        if all(value is not None for value in current_readings.values()):
            complete_reading = (datetime.now(), current_readings["x"], current_readings["y"], current_readings["z"])
            accelerometer_history.append(complete_reading)

def create_live_graph(dataframe):
    """Create a Plotly figure showing the accelerometer data over time."""
    figure = go.Figure()
    figure.add_trace(go.Scatter(x=dataframe["timestamp"], y=dataframe["x"], name="X-axis"))
    figure.add_trace(go.Scatter(x=dataframe["timestamp"], y=dataframe["y"], name="Y-axis"))
    figure.add_trace(go.Scatter(x=dataframe["timestamp"], y=dataframe["z"], name="Z-axis"))
    figure.update_layout(
        margin=dict(l=30, r=10, t=30, b=30),
        xaxis_title="Time",
        yaxis_title="Acceleration (g)"
    )
    return figure

def calculate_statistics(dataframe):
    """Calculate mean and standard deviation for each accelerometer axis."""
    return {
        "x_mean": dataframe["x"].mean(), "x_std": dataframe["x"].std(),
        "y_mean": dataframe["y"].mean(), "y_std": dataframe["y"].std(),
        "z_mean": dataframe["z"].mean(), "z_std": dataframe["z"].std(),
    }

def export_data_to_csv():
    """Export all collected accelerometer data to a timestamped CSV file."""
    with data_lock:
        all_readings = list(accelerometer_history)
    
    if not all_readings:
        return None
        
    dataframe = pd.DataFrame(all_readings, columns=["timestamp", "x", "y", "z"])
    timestamp_string = generate_timestamp_filename()
    csv_file_path = os.path.join(data_output_folder, f"accelerometer_data_{timestamp_string}.csv")
    
    # Format timestamps for CSV export
    formatted_dataframe = dataframe.copy()
    formatted_dataframe["timestamp"] = [timestamp.strftime("%Y-%m-%d %H:%M:%S") for timestamp in formatted_dataframe["timestamp"]]
    formatted_dataframe.to_csv(csv_file_path, index=False)
    
    print("Data exported to CSV:", csv_file_path)
    return csv_file_path

def arduino_cloud_connection_thread():
    """Background thread that continuously connects to Arduino Cloud and collects data."""
    current_readings = {"x": None, "y": None, "z": None}
    
    while not shutdown_signal.is_set():
        try:
            # Establish connection to Arduino Cloud
            cloud_client = ArduinoCloudClient(
                device_id=DEVICE_ID,
                username=DEVICE_ID,
                password=SECRET_KEY,
                sync_mode=True
            )
            
            def register_variable_callback(variable_names, callback_function): 
                for variable_name in variable_names: 
                    cloud_client.register(variable_name, value=None, on_write=callback_function)

            # Register callbacks for each axis
            register_variable_callback(x_axis_variables, lambda client, value: add_complete_reading("x", value, current_readings))
            register_variable_callback(y_axis_variables, lambda client, value: add_complete_reading("y", value, current_readings))
            register_variable_callback(z_axis_variables, lambda client, value: add_complete_reading("z", value, current_readings))

            print("Establishing connection to Arduino IoT Cloud...")
            cloud_client.start()
            print("Successfully connected to Arduino Cloud (Live monitoring mode)")

            # Continuously poll for new data
            while not shutdown_signal.is_set():
                cloud_client.update()
                time.sleep(0.05)  # Poll at 20 Hz
                
        except Exception as connection_error:
            print("Connection lost, attempting to reconnect in 3 seconds:", connection_error)
            time.sleep(3)

def start_live_dashboard(server_port=8053, display_window=max_samples_in_graph):
    """Launch the live accelerometer data visualization dashboard."""
    dashboard_app = Dash(__name__)
    dashboard_app.title = "SIT225 Week 8.2C — Live Accelerometer Data Stream"

    dashboard_app.layout = html.Div(
        style={"fontFamily": "Arial", "maxWidth": "1000px", "margin": "20px auto"},
        children=[
            html.H2("SIT225 Week 8.2C — Live Accelerometer Data Stream"),
            html.Div(f"Displaying the most recent {display_window} data points (sliding window)"),
            dcc.Graph(id="accelerometer-graph", style={"height": "500px"}),
            html.Pre(id="statistics-display", style={"fontFamily": "monospace", "whiteSpace": "pre-wrap"}),
            html.Div(id="collection-status", style={"color": "#555"}),
            dcc.Interval(id="update-timer", interval=500, n_intervals=0),  # Refresh every 500ms
        ]
    )

    @dashboard_app.callback(
        [Output("accelerometer-graph", "figure"),
         Output("statistics-display", "children"),
         Output("collection-status", "children")],
        Input("update-timer", "n_intervals"),
        prevent_initial_call=False
    )
    def update_dashboard(_):
        global previous_graph_figure, total_samples_collected
        
        with data_lock:
            recent_readings = list(accelerometer_history)[-display_window:]
            total_samples = len(accelerometer_history)
            
        if recent_readings:
            # Create dataframe from recent readings
            readings_dataframe = pd.DataFrame(recent_readings, columns=["timestamp", "x", "y", "z"])
            readings_dataframe["timestamp"] = [timestamp.strftime("%H:%M:%S") for timestamp in readings_dataframe["timestamp"]]
            
            # Generate live graph
            live_graph = create_live_graph(readings_dataframe)
            previous_graph_figure = live_graph
            
            # Calculate and display statistics
            statistics_data = calculate_statistics(readings_dataframe.drop(columns=["timestamp"]))
            statistics_text = "\n".join(f"{stat_name:8s}: {stat_value:.5f}" for stat_name, stat_value in statistics_data.items())
        else:
            live_graph = previous_graph_figure if previous_graph_figure else go.Figure()
            statistics_text = "Waiting for accelerometer data..."
            
        status_message = f"Total samples collected: {total_samples}"
        
        # Automatically save data periodically
        if total_samples and total_samples % auto_save_interval == 0:
            export_data_to_csv()
            
        return live_graph, statistics_text, status_message

    # Start the dashboard server
    print(f"Launching dashboard at http://127.0.0.1:{server_port}")
    dashboard_app.run(host="127.0.0.1", port=server_port, debug=False)

# Main execution
if __name__ == "__main__":
    print("Data will be saved to:", os.path.abspath(data_output_folder))
    
    # Start the Arduino Cloud connection in a background thread
    cloud_thread = threading.Thread(target=arduino_cloud_connection_thread, daemon=True)
    cloud_thread.start()
    
    # Launch the live dashboard
    start_live_dashboard()


Data will be saved to: /Users/adityasuhag/SIT225_2024T2/week 8 content/task objective/week8C_data
Establishing connection to Arduino IoT Cloud...
Launching dashboard at http://127.0.0.1:8053


Successfully connected to Arduino Cloud (Live monitoring mode)
